In [1]:
import os
import ast

import pandas as pd
from fastai.vision import *

In [2]:
path_csv = os.path.join(os.path.abspath('.'), 'data', 'good_cats.csv')
df = pd.read_csv(path_csv)

In [3]:
breeds = df.breed.unique()
for breed in breeds:
    one_breed = df[(df.breed == breed)]
    path_breed_csv = os.path.join(os.path.abspath('.'), 'images', breed + '.csv')
    with open(path_breed_csv, 'w') as f:
        for ind in one_breed.index:
            lst_url = ast.literal_eval(one_breed.loc[ind].photos)
            f.write(lst_url[0] + '\n')

In [4]:
path = Path(os.path.join(os.path.abspath('.'), 'images'))
for breed in breeds:
    folder = breed
    file = breed + '.csv'
    dest = path/folder
    dest.mkdir(parents=True, exist_ok=True)
    download_images(path/file, dest, max_pics=500)

In [5]:
for breed in breeds:
    verify_images(path/breed, delete=True, max_size=500)

In [6]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)

In [7]:
learn = create_cnn(data, models.resnet34, metrics=error_rate)

In [8]:
learn.fit_one_cycle(2)

epoch,train_loss,valid_loss,error_rate
1,2.815784,2.369524,0.649271
2,2.433556,2.229602,0.614263


In [34]:
learn.export()